# Detekce obličeje pomocí termokamery

Cvičení je zaměřené na práci s [termokamerami](https://en.wikipedia.org/wiki/Thermographic_camera). Termokamera je zařízení, které snímá obraz v IR spektru, podobně jako RGB kamery, které snímají ve viditelném spektru. Výstupem termokamery je `Radiometric image`, ve kterém je v každém pixelu zakódována hodnota nasnímané teploty v proprietárním formátu. Dnešní pokročilejší software dovoluje exportovat nasnímané obrázky ve formátu `csv`, ve kterém jsou umístěné skutečné hodnoty teplot ve škále pro nás známé.


Termokamery jsou aktivně používané v průmyslu, např. pro kontrolu zahřívání elektických součástek nebo kotrolu vadností solárních panelů pomocí dronů.

![](images/solar.png)

## Parametry pro snímání termokamerou

V souvislosti s termokamerou rozlišujeme u měřených materiálů 3 hlavní vlastnosti: **emisivitu**, **transmisivitu** a **reflexivitu**. Pomocí termokamery měříme množství energie, kterou daný materiál vyzařuje, odráží a propouští.

- **emisivita**: schopnost materiálu vyzařovat absorbované teplo do okolí
- **reflexivita**: schopnost materiálu odrážet okolní teplo
- **transmisivita**: Schopnost materiálu propouštět teplo

Tyto vlastnosti jsou bezrozměrné veličiny nabývajících hodnot $\langle0;1\rangle$ a jejich součet je pro každý materiál 1. Tedy celková energie dopadající na čidlo termokamery je součet energie, kterou těleso vyzařuje, odráží a propouští.

Obyčejně mají materiály vysokou emisivitu a nízkou reflexivitu a transitivitu. Budeme zkoumat atypické materiály s vysokou reflexivitou (chovají se jako zrcadlo) a transmitivitou (jeví se jako průhledné).

## Tepelné vlastnosti žárovek

Nejčastěji nacházíme 3 druhy žárovek - žárovky **klasické**, **zářivky** a **LED**.

#### Klasické žárovky
Princip klasické žárovky je založený na zahřívání tenkého, nejčastěji wolframového vodiče elektrickým proudem, který jím protéká. V dnešní době jsou nahrazovány jinými typy žárovek kvůli malé efektivitě (většina spotřebované elektřiny - až 95% - je přeměněna na tepelnou energii a zbytek na světlo). Životnost těchto žárovek se pohybuje od  800 do 1000 hodin.

#### Kompaktní zářivky
Mají životnost od 6000 do 16000 hodin a jejich spotřeba je o 80 % nižší než u klasických žárovek. Základem zářivky je těleso, které se skládá nejčastěji z dlouhé skleněné trubice obsahující žhavící elektrody. Je naplněna rtuťovými parami a argonem. Uvnitř trubice nastává doutnavý výboj, který z velké části září v ultrafialové oblasti. Stěny trubice jsou pokryty luminoforem, který UV záření absorbuje a sama září ve viditelné oblasti.

#### LED žárovky
V součastnosti jsou nejoblíbenějším typem žárovek. Mají na stejnou světelnost nejnižší spotřebu a vydrží svítit až padesát tisíc hodin. Ve spodní části LED žárovky se nachází elektronika, která mění střídavý proud na stejnosměrný (který LED vyžaduje). U některých typů LED žárovek lze ve spodní části také nalézt elektroniku na ovládání barvy a jasu pomocí dálkového ovládání. Nad elektronikou se nachází pouze samotná LEDka zakrytá difuzním obalem.

![](images/bulbs.jpeg)

### Import knihoven a konfigurace

In [ ]:
from collections import OrderedDict
import numpy as np
import cv2
from improutils import *

### Pomocné funkce
Z následujících funkcí je potřeba vybírat ty vhodné pro splnění úkolu.

Třída pro práci s popisnými charakteristikami tvarů a jednotlivé dílčí funkce. **Pozor, že funkce níže jako parametr berou obrázek a NE konturu!** 

- [`roundness(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/recognition/image_features.py#L38)
- [`aspect_ratio(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/recognition/image_features.py#L48)
- [`convexity(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/recognition/image_features.py#L57)
- [`solidity(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/recognition/image_features.py#L68)
- [`compactness(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/recognition/image_features.py#L79)
- [`extent(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/recognition/image_features.py#L89)

Další zajímavé funkce
- [`crop_by_bounding_rect(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/preprocessing/preprocessing.py#L99)
- [`crop_contour(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/recognition/image_features.py#L89)
- [`contour_to_image(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/preprocessing/contours.py#L5)
- a všechny ostatní, co naleznete v [`improutils`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package)

Normalizace obrázku a převedení do BGR

In [ ]:
def normalize_image(img):
    scaled = ((img - img.min()) * (1/(img.max() - img.min()) * 255)).astype('uint8')
    return cv2.cvtColor(scaled, cv2.COLOR_GRAY2BGR)

Načtení CSV souboru do obrázku

In [ ]:
def load_csv(file_path):
    lines = ""
    with open(file_path, "r") as f:
        lines = f.readlines()
        lines = [line.replace(",", ".") for line in lines]
    with open(file_path, "w") as f:
        f.writelines(lines)
    return np.genfromtxt(file_path, delimiter=';')[:, :-1]

---

### Úkol
Detekujte obličej na termosnímku, použijte znalosti klasifikace pomocí tvarových chrakteristik z předchozího cvičení.

#### 1) Získejte termosnimek s obličejem
Přejděte k učitelskému počítači a nechte se vyfotit termokamerou. Snímky, které vám budou zaslány na e-mail, si stáhněte a jeden z nich načtěte.

Pro zjednodušení zobrazování lze použít pomocnou funkci `normalize_image`, která převede csv obrázek na BGR obrázek.

In [ ]:
file_path = '' ###
img_csv = ...(file_path) ###

plot_images(...(img_csv)) ###

#### 2) Segmentace obličeje
Vymyslete způsob, jak se dá segmentovat obličej v termosnímku. Využijte znalosti o **teplotě** lidské kůže (NEPOŽÍVEJTE normalizované šedotónové obrázky s hodnotami jasu, neboť to **nebude fungovat obecně**!). Zkuste experimentovat s parametry na více snímkách s vámi a vaším kolegou.

In [ ]:
bin_mask = ...(img_csv, ?, ?) ###
plot_images(bin_mask)

#### 3) Získejte vlastnosti segmentovaných objektů
Získejte kontury segmentovaných objektů, a odfiltrujte šum, např. pomocí plochy kontury. Dejte pozor na to, že obličej  může mít na různých snímkách různé rozměry.

U získaných kontur spočtěte popisné tvarové charakteristiky a vypište jejich hodnoty (viz [Pomocné funkce](#Pomocné-funkce)). Použijte znalosti z minulého cvičení.

In [ ]:
contours_bin, _, conts = ...(bin_mask, ?) ###
faces = apply_mask(img_csv, contours_bin) ###
plot_images(faces)

In [ ]:
# výpis hodnot tvarových charakteristik
for contour in conts:
    print('Délka:        ', len(contour))
    print('Plocha:       ', cv2.contourArea(contour))
    
    ### další charakteristiky
    
    
    
    
    
    
    
    
    print('--------------------------------')

#### 4) Vyberte ručně významné vlastnosti a jejich hodnoty na odlišení obličeje od ostatních objektů
Vytvořte proměnné pro prahy (limity) hodnot tvarových charakteristik použitelných pro filtraci kontur. Vytvořte vhodné podmínky pro filtraci kontur.

In [ ]:
# Příklad, jak by mohly vypadat proměnné pro prahy (limity)
..._thresh = ###
..._thresh = ###
### ...

filtered_conts = []
for contour in conts:
    face_candidate = ... ### vytvoření obrázku z kontury
    
    # vhodný počet podmínek
    if ... and ...:
        filtered_conts.append(contour)
        
print('Počet nalezených obličejů je ' + str(len(filtered_conts)))

#### 5) Udělejte vizualizaci výsledné detekce
Použijte aproximace obdélníkem nebo elipsou pro demonstrační účely ([ukázka](https://docs.opencv.org/3.1.0/dd/d49/tutorial_py_contour_features.html)).

In [ ]:
demo_image = ...(img_csv) ###
for cont in filtered_conts:
    ### nakreslení obdélníku/elipsy do obrázku
    
    
plot_images(demo_image)

#### 6) Vyzkoušejte vás detekční algoritmus na referenčním obrázku

In [ ]:
filenames = ['data/1.csv', 'data/2.csv']

### celý algoritmus v jednom včetně vizualizace konečného výsledku
